# Skript for retreving tumblr posts with API.
## Based on [tumblr-scraper](https://github.com/boilingpenguin/tumblr-scraper).

In [3]:
import pytumblr
import unicodecsv as csv
from unidecode import unidecode
import calendar
import time

In [ ]:
# Tumblr credentials:
tumblr = pytumblr.TumblrRestClient(
  '<...>'
)

"""Search parameters:

Tumblr's API only supports searching for a single tag at a time.
Filter options are text, html, or raw.
Before - only search for posts before this time - in seconds past the epoch.
You can use calendar.timegm(time.gmtime()) to use the current epoch.
"""

before = calendar.timegm(time.gmtime())

TAG: str = 'essstörung'
FILTER: str = 'raw'
OUTPUT_PATH: str = '../data/'
FILE_PATH = OUTPUT_PATH + 'TumblrSearch-' + str(before) + '-' + TAG + '.csv'

with open(FILE_PATH, mode='ab') as results_file:
		results_writer = csv.writer(results_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
		results_writer.writerow(["timeStamp", "URL", "blogName", "title", "tags", "body"])


print("Searching Tumblr for 100 posts tagged \'" + TAG + "\'")

j = 1

"""It is important to pay attention to the number of batches chosen here.
After a certain rate, users will receive an API call limit.
Using 20 batches brings around 450 posts.
"""

while j < 20:
	# Run the tag search and snag the results
	search_results = tumblr.tagged(TAG, filter=FILTER, before=before)
	print("Batch " + str(j) + " of 20")
	print(str(len(search_results)) + " results retreived")
	for i in search_results:
		blog_name = (i)['blog_name']
		date = (i)['date']

		url = (i)['post_url']

		try:
			title = (i)['title']
		except:
			title = "Couldn't Get Title"
		try:
			tags = (i)['tags']
		except:
			tags = "Couldn't Get tags"
		try:
			body = (i)['body']
		except:
			body = "Couldn't Get Post Body"
		with open(FILE_PATH, mode='ab') as results_file:
			results_writer = csv.writer(results_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
			try:
				results_writer.writerow([date, unidecode(url), blog_name, title, tags, body])
				print("Successfully Wrote Row for post from " + date)
			except:
				print("Error Writing Row")
	#Next we make note of the oldest timestamp
		oldest_time = (i)['timestamp']
	before = oldest_time
	j = j + 1
print("Finished!")
print("Saved CSV to " + FILE_PATH)